In [1]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time
from collections import namedtuple
import os
import xml.etree.ElementTree as ET

In [2]:
settings = {
    'queries_xml': '/home/subject/Downloads/web2008_adhoc.xml', # path to queries xml
    'relevance_xml': '/home/subject/Downloads/or_relevant-minus_table.xml', # path to relevance xml
    'collection_dir': '/home/subject/Documents/informational retrieval/all_info2/', # folder with contents of all_info.zip
    'pagerank_json' : '/home/subject/Documents/informational retrieval/pagerank.json' # json file with pageranks
}

In [3]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'timeout': 360, 'maxsize': 25}])

In [4]:
if not es.indices.exists(index='byweb'):
    es.indices.create(index='byweb')

In [5]:
index_settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            },
            'stem_content': {
                'type': 'text'
            },
            'title': {
                'type': 'text'
            },
            'id': {
                'type': 'keyword'
            },
            'url': {
                'type': 'keyword'
            },
            'pagerank': {
                'type': 'rank_feature'
            }
        }
    },
    'settings': {
       'analysis': {
            'analyzer': {
                'white_lover': {
                    'tokenizer': 'white_20',
                    'filter': [
                        'lowercase'
                    ]
                }
            },
            'tokenizer': {
                'white_20': {
                    'type': 'whitespace'
                }
            },
        }
    }
}

In [6]:
def recreate_index():
    es.indices.delete(index='byweb')
    es.indices.create(index='byweb', body=index_settings)

In [7]:
%time
recreate_index()

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 16.9 µs


In [8]:
# Retrieval of pagerank
def get_rank_jsons():
    file = settings['pagerank_json']
    with open(file) as f:
        l = json.load(f)
        return dict(l)

In [9]:
pagerank_dict = get_rank_jsons()

In [10]:
from urllib.parse import urlparse
def get_pagerank_by_url(url):
    domain = urlparse(url).netloc
    # lonely vertex -> pagerank is small
    return pagerank_dict.get(domain, 1e-9)

In [11]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

def es_actions_generator(docs):
    for doc in os.listdir(docs):
        doc = os.path.join(docs, doc)
        with open(doc) as d:
            data = json.load(d)
            for document in data:
                document['pagerank'] = get_pagerank_by_url(document["url"])
                yield create_es_action('byweb', document['id'], document)

In [12]:
generator = es_actions_generator(settings['collection_dir'])
for ok, result in tqdm(parallel_bulk(es, generator, queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)

Время работы процедуры выше: ~5 минут.

Если посмотрим сюда: "http://localhost:9200/_stats/indexing,store?pretty " и найдем:

`indices.byweb.total.store.size_in_bytes`

увидим, что размер индекса равен ~3 Гб. Размер информации, запихнутой в индекс, ~4.1 Гб. 

Размер индекса с Pagerank - ~4 Гб.

(Конечно, все это можно было сделать программно, но посмотреть на один показатель быстрее ручками.)

In [13]:
Query = namedtuple('Query', ['query_id', 'text', 'relevant'])

In [14]:
def parse_query(query):
    text = query.find('{http://www.romip.ru/data/adhoc}querytext').text
    query_id = query.attrib['id']
    return Query(query_id=query_id, text=text, relevant=[])


def extract_queries():
    filename = settings['queries_xml']
    with open(filename) as f:
        tree = ET.parse(filename)
        root = tree.getroot()
        tasks = root.findall('{http://www.romip.ru/data/adhoc}task')
        qs = list(map(parse_query, tasks))
        queries = dict()
        for x in qs:
            queries[x.query_id] = x
        return queries

In [15]:
def extract_relevance():
    queries_dict = extract_queries()
    def parse_relevance(query):
        query_id = query.attrib['id']
        q = queries_dict[query_id]
        for doc in query.findall('./{http://www.romip.ru/common/merged-results}document'):
            if doc.attrib['relevance'] == 'vital':
                q.relevant.append(doc.attrib['id'])
        return q

    filename = settings['relevance_xml']
    with open(filename) as f:
        tree = ET.parse(filename)
        root = tree.getroot()
        tasks = root.findall('{http://www.romip.ru/common/merged-results}task')
        return list(map(parse_relevance, tasks))

In [16]:
queries = extract_relevance()
# important: -38 queries
queries = list(filter(lambda q: q.relevant, queries))
print(len(queries))

495


In [17]:
def search(query, result_size=20):
    """
    Use this function for search.
    """
    
    query = {
        'query': {
             'bool': {
                'should': 
                    {
                        'match': {
                            'content': query.text.lower()
                        }
                    }
             }
        }
    }

    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

def pretty_print_result(search_result):
    res = search_result['hits']
    print(f'Total documents: {res["total"]["value"]}')
    for hit in res['hits']:
        print(f'Doc {hit["_id"]}, score is {hit["_score"]}')
                  
def get_doc_by_id(doc_id):
    return es.get(index='byweb', id=doc_id)['_source']

Статистические показатели.

In [18]:
# returns [precision, recall, recall-]. Last one is for educational purpose only.
def stats_at(k, query, query_result):
    relevant = query.relevant
    result_at_k = query_result[:k]

    good = set(result_at_k).intersection(relevant)
    return [len(good) / k, len(good) / len(relevant), len(good) / min(k, len(relevant))]

In [19]:
# returns [ap, ap-r, ap-k, ap-]. Last 3 are for educational purpose only
def get_ap_at(k, query, query_result):
    s = 0
    n = 0
    relevant = query.relevant
    result_at_k = query_result[:k]

    for i in range(len(result_at_k)):
        result = result_at_k[i]
        if result in relevant:
            s += stats_at(i + 1, query, result_at_k)[0]
            n += 1
    
    return [s / n if n != 0 else 0, s / len(relevant), s / k, s / min(k, len(relevant))]

In [20]:
# returns [map, map-r, map-k, map-]. Last 3 are for educational purpose only
def get_map_at(k, queries, q_results):
    s1 = 0
    s2 = 0
    s3 = 0
    s4 = 0
    for query, query_result in zip(queries, q_results):
        ap = get_ap_at(k, query, query_result)
        s1 += ap[0]
        s2 += ap[1]
        s3 += ap[2]
        s4 += ap[3]
    return [s1 / len(queries), s2 / len(queries), s3 / len(queries), s4 / len(queries)]

In [21]:
def get_queries_results(qs, search_fun=search, k=20):
    # important: k = max(k, len(q.relevant)), because for r-precision we may need more than k.
    return [search_fun(q, max(k, len(q.relevant))) for q in qs]

In [22]:
def get_queries_stats(search_fun, k=20):
    results = get_queries_results(queries, search_fun, k)
    
    precision = 0
    recall = 0
    recall_dash = 0
    r_precision = 0
    for q, r in zip(queries, results):
        stats = stats_at(k, q, r)
        precision += stats[0]
        recall += stats[1]
        recall_dash += stats[2]
        r_precision += stats_at(len(q.relevant), q, r)[0]
    
    queries_size = len(queries)
    print("Средняя точность на уровне k=20 по всем запросам:", precision / queries_size)
    print("Средняя полнота на уровне k=20 по всем запросам:", recall / queries_size)
    print("Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам:", recall_dash / queries_size)
    print("Средняя R-точность по всем запросам:", r_precision / queries_size)
    map_at = get_map_at(k, queries, results)
    print("MAP на уровне k=20:", map_at[0])
    print("MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20:", map_at[1])
    print("MAP-k (в знаменателе AP - k) на уровне k=20:", map_at[2])
    print("MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20:", map_at[3])

In [23]:
get_queries_stats(search)

Средняя точность на уровне k=20 по всем запросам: 0.33878787878787886
Средняя полнота на уровне k=20 по всем запросам: 0.22243745709461563
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.39414125273704026
Средняя R-точность по всем запросам: 0.27752064450795616
MAP на уровне k=20: 0.505302904265171
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.15013623269259188
MAP-k (в знаменателе AP - k) на уровне k=20: 0.26675960103013524
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.2939187654754581


In [24]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries)

3 loops, best of 3: 5.31 s per loop


Заметки:

Query(query_id='arw53730', text='Ремолан', relevant=['980017']) []
потому что mystem, ремолать?

Зачем нужны дэш-характеристики:

Query(query_id='arw50384', text='гизметео', relevant=['587130', '856298', '239338', '239387', '239377', '687059', '261338', '325277', '72835', '1285231', '223037', '1382550', '37733', '92274', '92282', '1420731', '689932', '1303050', '963503', '602944', '1372162', '256386', '1427662', '1464330', '1357789']) ['1420731', '691030'] 1.0

In [25]:
# Ячейка для лемматизации запроса; совпадает с лемматизацией для коллекции.

stop_words = {'г', '©'}
def get_stop_words(files):
    for file in files:
        with open(file) as f:
            for word in f:
                stop_words.add(word.split()[0])

get_stop_words(['../extractor/stopwords/english', '../extractor/stopwords/russian'])

def is_not_stop_word(d):
    return not getLexOrText(d) in stop_words


normal_word = re.compile('^[A-Za-z0-9Ѐ-ӿ]*$')
def is_normal_word(d):
    return normal_word.match(d['text']) is not None


def getText(d):
    return d['text'].lower()

def getLexOrText(d):
    if 'analysis' not in d or not d['analysis']:
        return getText(d)
 
    analysis = d['analysis'][0]
    return analysis['lex'] if 'lex' in analysis else getText(d)


from pymystem3 import Mystem
m = Mystem()

def lemmatize(query_text):
    result = m.analyze(query_text)

    result = list(filter(bool, result))
    result = list(filter(lambda x: 'analysis' in x or is_normal_word(x), result))
    json_results = list(filter(is_not_stop_word, result))

    lexed_content = " ".join(list(map(getLexOrText, json_results)))
    return lexed_content

In [26]:
def l_search(query, result_size=20):
    query_text = lemmatize(query.text)
    query = {
        'query': {
             'bool': {
                'should': 
                    {
                        'match': {
                            'stem_content': query_text
                        }
                    }
            }
        }
    }
    
    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

In [27]:
get_queries_stats(l_search)

Средняя точность на уровне k=20 по всем запросам: 0.39818181818181864
Средняя полнота на уровне k=20 по всем запросам: 0.2712074457516083
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.46816114139549775
Средняя R-точность по всем запросам: 0.33883150733062384
MAP на уровне k=20: 0.5535287107311269
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.1856681552625555
MAP-k (в знаменателе AP - k) на уровне k=20: 0.3158996055921541
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.3534022369404635


In [28]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries, l_search)

3 loops, best of 3: 5.51 s per loop


Pagerank

In [29]:
def lp_search(query, result_size=20, saturation={}):
    query_text = lemmatize(query.text)
    query = {
        'query': {
             'bool': {
                'should': [
                    {
                        'match': {
                            'stem_content': {
                                'query': query_text,
                            }
                        }
                    },
                    {
                        'rank_feature': {
                            'field': 'pagerank',
                            'saturation': saturation,
                        }
                    }
                ]
            }
        }
    }

    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

In [30]:
get_queries_stats(lp_search)

Средняя точность на уровне k=20 по всем запросам: 0.3941414141414144
Средняя полнота на уровне k=20 по всем запросам: 0.27245669655357124
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.4660861082987928
Средняя R-точность по всем запросам: 0.33465876874214934
MAP на уровне k=20: 0.5476735995697807
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.18252628043917526
MAP-k (в знаменателе AP - k) на уровне k=20: 0.3105606037092688
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.3463676750901122


In [31]:
from functools import partial
saturation = {'pivot': 1e-6}
get_queries_stats(partial(lp_search, saturation=saturation))

Средняя точность на уровне k=20 по всем запросам: 0.39848484848484883
Средняя полнота на уровне k=20 по всем запросам: 0.2718922305790838
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.46903124594981277
Средняя R-точность по всем запросам: 0.33831016924139967
MAP на уровне k=20: 0.5513408942425277
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.1854266741485779
MAP-k (в знаменателе AP - k) на уровне k=20: 0.31559249897822916
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.353160124055841


In [32]:
from functools import partial
saturation = {'pivot': 1e-8}
get_queries_stats(partial(lp_search, saturation=saturation))

Средняя точность на уровне k=20 по всем запросам: 0.3984848484848489
Средняя полнота на уровне k=20 по всем запросам: 0.2719752747075097
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.4690564984750653
Средняя R-точность по всем запросам: 0.3386345237853433
MAP на уровне k=20: 0.5529794553779379
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.18566011409832686
MAP-k (в знаменателе AP - k) на уровне k=20: 0.31604154047420285
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.3536093762169382


In [33]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries, lp_search)

3 loops, best of 3: 6.84 s per loop


pagerank слегка ухудшил результат. Заметим, что если поиграться с pivot, можно добиться небольшого улучшения даже относительно l_search, но ничего значительного не получим.

In [34]:
def lt_search(query, result_size=20):
    query_text = lemmatize(query.text)
    query = {
        'query': {
             'bool': {
                'should': [
                    {
                        'match': {
                            'stem_content': {
                                'query': query_text,
                                'boost': 10
                            }
                        }
                    },
                    {
                        'match': {
                            'title': {
                                'query': query.text
                            }
                        }
                    }
                ]
            }
        }
    }

    result = es.search(index='byweb', body=query, size=result_size)
    return list(map(lambda x: x['_id'], result['hits']['hits']))

In [35]:
get_queries_stats(lt_search) # 10

Средняя точность на уровне k=20 по всем запросам: 0.4075757575757578
Средняя полнота на уровне k=20 по всем запросам: 0.27798909725396764
Средняя полнота-дэш (в знаментеле стоит k, если релевантных документов больше, чем k) на уровне k=20 по всем запросам: 0.4801656277549889
Средняя R-точность по всем запросам: 0.35278865631007517
MAP на уровне k=20: 0.5680905712742932
MAP-r (в знаменателе AP - количество всех релевантных документов для запроса) на уровне k=20: 0.19083621687895494
MAP-k (в знаменателе AP - k) на уровне k=20: 0.3251037497682118
MAP-дэш (в знаменателе AP - k, если релевантных документов больше k, иначе - количество всех релевантных документов для запроса) на уровне k=20: 0.36461177147359297


In [36]:
%%timeit -n 3 -r 3
_ = get_queries_results(queries, lt_search)

3 loops, best of 3: 6.35 s per loop
